# PyMathML

PyMathML is a Python package to create MathML expressions programatically.

MathML is extremely verbose; with PyMathML, concise, pythonic expressions are
converted to valid, well-formed MathML code. For example, the following snippet

In [ ]:
import xml.etree.ElementTree as ET
from pymathml import *

In [ ]:
a = Identifier('a')
b = Identifier('b')
expr = a**2+2*a*b+b**2

defines the mathematical expression ``a²+2ab+b²``. PyMathML then produces the following MathML code

In [ ]:
ET.dump(expr.to_mml())